In [1]:
import pandas as pd
import random
import os
import ast

# Master Movie Data Manipulation Notebook

How this notebook could work is that we iterate/clean on the 'meta' csv in pandas, and then save it in the repo after we have added the new/corrected/cleaned data

In [2]:
#### change pathname to your clean dataset
meta = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/movies_metadata.csv')
#meta.head(2).T

/Users/nickparker/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(meta.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [4]:
### Adding profit and use of Budget Columns
meta['profit'] = meta['revenue'] - pd.to_numeric(meta['budget'],errors='coerce')
meta['useOfBudget'] = meta['revenue'] / (pd.to_numeric(meta['budget'],errors='coerce') + 000.1)

In [5]:
##Adding production company
#Added Production Companies as a trend
productionCompanies1 = []
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies1.append(ast.literal_eval(meta['production_companies'][rowNum])[0]['name'])
    except:
        productionCompanies1.append('nan')
        
productionCompanies2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies2.append(ast.literal_eval(meta['production_companies'][rowNum])[1]['name'])
    except:
        productionCompanies2.append('nan')
        
meta['ProductionCo1'] = productionCompanies1
meta['ProductionCo2'] = productionCompanies2

In [6]:
#meta[['title','ProductionCo1','ProductionCo2']]

In [7]:
##Adding genre
genre1 = []
for rowNum in range(meta.shape[0]):
    try: 
        genre1.append(ast.literal_eval(meta['genres'][rowNum])[0]['name'])
    except:
        genre1.append('nan')
        
genre2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre2.append(ast.literal_eval(meta['genres'][rowNum])[1]['name'])
    except:
        genre2.append('nan')

genre3 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre3.append(ast.literal_eval(meta['genres'][rowNum])[2]['name'])
    except:
        genre3.append('nan')
meta['genre_1'] = genre1
meta['genre_2'] = genre2
meta['genre_3'] = genre3

## Merging

In [8]:
## Don't run this cell unless you want to merge in ratings (big file)
#Also, update filepath
ratings = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/ratings.csv')
links = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/links.csv')

ratings = ratings.groupby('movieId').agg({'rating':['mean']}).reset_index() #gets average rating
ratings.columns = ['movieId','rating']
linkAvgRating = pd.merge(links,ratings,on='movieId',how='left')

#credits = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/credits.csv')
#credits.columns = ['cast','crew','movieId']
#linkRatCred = pd.merge(linkAvgRating,credits,on='movieId',how='left')

meta['imdbId'] = pd.to_numeric(meta['imdb_id'].str[2:])
meta = pd.merge(meta,linkAvgRating,on='imdbId',how='left')

## Making Flops dataframe

In [10]:
flop = meta.loc[((meta['revenue'] > 0) & pd.to_numeric(meta['budget'],errors='coerce' != 0))]
flop['flop'] = 0
flop.loc[(flop['profit'] < 0),'flop'] = 1
flop.shape

/Users/nickparker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/nickparker/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(5381, 36)

## Making the df longer based on genre

In [11]:
flop.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'profit', 'useOfBudget', 'ProductionCo1',
       'ProductionCo2', 'genre_1', 'genre_2', 'genre_3', 'imdbId', 'movieId',
       'tmdbId', 'rating', 'flop'],
      dtype='object')

In [12]:
#metaTest = meta[['id','genre_1','genre_2','genre_3']]
#pd.wide_to_long(metaTest, stubnames='genre', i=['imdb_id'], j='genreNo')
#pd.melt(metaTest,id_vars=['imdb_id'],var_name='metrics', value_name='values')
long_df = pd.wide_to_long(flop.reset_index(), 
                          i=['index'], 
                          j='genreNo',
                          stubnames='genre',
                          sep="_")
flop.shape

(5381, 36)

In [13]:
long_df.shape

(16143, 34)

In [17]:
long_df.reset_index(inplace=True)

display = long_df[['title','id','genreNo','genre']]

display.sort_values('id').head(10)

,title,id,genreNo,genre
6317,"Lock, Stock and Two Smoking Barrels",100,2,Crime
11698,"Lock, Stock and Two Smoking Barrels",100,3,nan
936,"Lock, Stock and Two Smoking Barrels",100,1,Comedy
5892,The Saint,10003,2,Action
11273,The Saint,10003,3,Romance
511,The Saint,10003,1,Thriller
9956,Dumb and Dumber To,100042,2,nan
4575,Dumb and Dumber To,100042,1,Comedy
15337,Dumb and Dumber To,100042,3,nan
2555,Cursed,10012,1,Horror


/Users/nickparker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
flop.shape

(5381, 36)

# Export

#### When you are done with the data manipulation, save as csv in our manipulated data folder

In [9]:
export_csv = meta.to_csv (r'movie_dataset_edited/meta.csv', header=True)
#Don't forget to add '.csv' at the end of the path

In [37]:
export_csv = flop.to_csv (r'movie_dataset_edited/flopDataSet.csv', header=True)


In [ ]:
export_csv = flop.to_csv (r'movie_dataset_edited/flopDataSet.csv', header=True)
